In [1]:
!pip install transformers sentencepiece 

     |████████████████████████████████| 2.9 MB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 31.6 MB/s 
     |████████████████████████████████| 3.3 MB 40.9 MB/s 
     |████████████████████████████████| 636 kB 47.6 MB/s 
     |████████████████████████████████| 895 kB 43.6 MB/s 
     |████████████████████████████████| 56 kB 4.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
import random
from tqdm.auto import tqdm, trange
import os
from sklearn.model_selection import train_test_split

pd.options.display.max_colwidth = -1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  # Remove the CWD from sys.path while we load stuff.


In [6]:
df = pd.read_csv('drive/MyDrive/elon_musk_dataset/short_answer_dialog_dataset.csv').dropna()
print(df.shape)
df.head()

(668, 2)


,context,answer
0,Welcome back.,Here we go again.
1,Great to see you and congratulations.,Thank you.
2,"You will never forget what is going on in the world when you think about when your child is born. You will know for the rest of this child’s life, you were born during a weird time.",That’s for sure.
3,That is for sure. Probably the weirdest that I can remember.,"Yeah. Yeah, and he was born on May the fourth."
4,"That’s hilarious, too.",Yeah.


# Обучение
Поскольку данных очень мало, обучение будет происходить без валидации. Оценка будет произведена глазами на тех же данных.  
Конечно, этот подход плохой, но как бейзлайн можно попробовать.

In [7]:
raw_model = 'google/t5-v1_1-base' 
model = T5ForConditionalGeneration.from_pretrained(raw_model).cuda();
tokenizer = T5Tokenizer.from_pretrained(raw_model)

Downloading:   0%|          | 0.00/605 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/945M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

In [8]:
batch_size = 16
report_steps = 200
epochs = 10

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
pairs = df.values.tolist()

In [9]:
model.train()
losses = []
for epoch in range(epochs):
    print('EPOCH', epoch)
    random.shuffle(pairs)
    for i in trange(0, int(len(pairs) / batch_size)):
        batch = pairs[i * batch_size: (i + 1) * batch_size]
        x = tokenizer([p[0] for p in batch], return_tensors='pt', padding=True).to(model.device)
        y = tokenizer([p[1] for p in batch], return_tensors='pt', padding=True).to(model.device)
        y.input_ids[y.input_ids == 0] = -100
        loss = model(
            input_ids=x.input_ids,
            attention_mask=x.attention_mask,
            labels=y.input_ids,
            decoder_attention_mask=y.attention_mask,
            return_dict=True
        ).loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses.append(loss.item())
        if i % report_steps == 0:
            print('step', i, 'loss', np.mean(losses[-report_steps:]))

EPOCH 0


  0%|          | 0/41 [00:00<?, ?it/s]

step 0 loss 15.017801284790039
EPOCH 1


  0%|          | 0/41 [00:00<?, ?it/s]

step 0 loss 14.129616419474283
EPOCH 2


...

step 0 loss 10.742886486053466
EPOCH 8


  0%|          | 0/41 [00:00<?, ?it/s]

step 0 loss 10.389183259010315
EPOCH 9


  0%|          | 0/41 [00:00<?, ?it/s]

step 0 loss 10.06748589038849


In [10]:
model.eval()

def answer(x, **kwargs):
    inputs = tokenizer(x, return_tensors='pt').to(model.device) 
    with torch.no_grad():
        hypotheses = model.generate(**inputs, **kwargs).cpu()
    return tokenizer.decode(hypotheses[0], skip_special_tokens=True)

In [11]:
sample = df.sample(5)
for i, row in sample.iterrows():
    print(row.context)
    print('answer:', row.answer)
    print('model: ', answer(row.context))
    print('---')

What would you buy if you could?
answer: Wait, is there something you think I should buy?
model:  . What would you do? What would you do? What would you do? What would
---
Okay. All right, so last thing on this, on Tesla, these new products. The truck, the Roadster, where are they?
answer: Yeah, I’m super-excited about the future —
model:  e, Tesla, Tesla. All right. All right. All right. All right.
---
But the initial thought on this virus, the real fear was that this was going to kill hundreds of thousands, if not millions of people, instantaneously in this country. It was going to do it very quickly if we didn’t hunker down, if we didn’t shelter in place, if we didn’t quarantine ourselves, or lock down. Do you think that the initial thought was a good idea based on the perception that this was going to be far more deadly than it turned out to be?
answer: Maybe. I think briefly.
model:  . It was a very scary virus. It was very dangerous. It was very dangerous
---
It’s not what we fea

In [12]:
new_model_name = 'drive/MyDrive/elon_musk_dialog_model'
model.save_pretrained(new_model_name)
tokenizer.save_pretrained(new_model_name)

('drive/MyDrive/elon_musk_dialog_model/tokenizer_config.json',
 'drive/MyDrive/elon_musk_dialog_model/special_tokens_map.json',
 'drive/MyDrive/elon_musk_dialog_model/spiece.model',
 'drive/MyDrive/elon_musk_dialog_model/added_tokens.json')